In [4]:
import pandas as pd
import pandas_profiling
import numpy as np
import json
from jinja2 import Environment, FileSystemLoader

In [2]:
%%javascript
require.config({
paths: {
d3: "https://d3js.org/d3.v5.min",
}
});

require(["d3"], function(d3) {
window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [ ]:
columns = ["symboling", "normalized-losses", "make", "fuel-type", "aspiration", "num-of-doors", "body-style", "drive-wheels", "engine-location", "wheel-base", "length", "width", "height",
           "curb-weight", "engine-type", "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower", "peak-rpm", "city-mpg", "highway-mpg", "price"]

df = pd.read_csv('imports-85.data', names=columns)

for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')

df.replace(to_replace=["na", "?", np.nan, "missing", "not available","n/a", "missing value"], value=np.nan, inplace=True)

In [ ]:
pandas_profiling.ProfileReport(df)

In [ ]:
columns = ["class", "age", "menopause", "tumor-size", "inv-nodes", 
           "node-caps", "deg-malig", "breast", "breast-quad", "irradiat"]

df = pd.read_csv('breast-cancer.data', names=columns)

df['deg-malig'].replace(to_replace={1: 'one', 2:'two', 3:'three'}, inplace=True)

df.replace(to_replace=["na", "?", np.nan, "missing", "not available","n/a", "missing value"], value=np.nan, inplace=True)


pandas_profiling.ProfileReport(df)

In [ ]:
columns = ["class","age","sex","histologic-type","degree-of-diffe","bone","bone-marrow","lung","pleura","peritoneum","liver","brain","skin","neck","supraclavicular","axillar","mediastinum","abdominal"]

df = pd.read_csv('primary-tumor.data', names=columns)

df.replace(to_replace=["na", "?", np.nan, "missing", "not available","n/a", "missing value"], value=np.nan, inplace=True)

for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')

df["class"].replace(to_replace= {1:"lung", 2:"head & neck", 3:"esophasus", 4:"thyroid", 5:"stomach", 6:"duoden & sm.int", 7:"colon", 8:"rectum", 9:"anus", 10:"salivary glands", 11:"pancreas", 12:"gallblader", 13:"liver", 14:"kidney", 15:"bladder", 16:"testis", 17:"prostate", 18:"ovary", 19:"corpus uteri", 20: "cervix uteri", 21:"vagina", 22:"breast"}, inplace=True)

df["age"].replace(to_replace ={1:"<30",2:"30-59",3:">60"}, inplace=True)
df["sex"].replace(to_replace ={1:"male",2:"female"}, inplace=True)
df["histologic-type"].replace(to_replace ={1.0:"epidermoid",2.0:"adeno", 3.0:"anaplastic"}, inplace=True)
df["degree-of-diffe"].replace(to_replace={1:"well", 2:"fairly", 3:"poorly"}, inplace=True)
df["bone"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["bone-marrow"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["lung"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["pleura"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["peritoneum"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["liver"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["brain"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["skin"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["neck"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["supraclavicular"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["axillar"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["mediastinum"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["abdominal"].replace(to_replace={1:"yes", 2:"no"}, inplace=True)
df["histologic-type"].unique()

In [ ]:
pandas_profiling.ProfileReport(df).to_file("../../../../Desktop/example.html")

In [ ]:
df = pd.DataFrame({'porta': ['duas', 'quatro', 'duas', 'quatro', 'quatro', 'duas', 'duas'],
                    'combustivel': ['gas', 'diesel', 'gas', 'gas', 'diesel', 'diesel', 'gas'],
                    'body-style': ['sedan', 'hatch', 'hatch', 'sedan', 'sedan', 'hatch', 'hatch']
                    })

In [ ]:
table_data_format(df)

In [3]:
def build_data(df):
    count = []
    considered_cols = []
    last_index = 0
    keys = {}

    df.replace(to_replace=["na", "?", np.nan, "missing", "not available",
                           "n/a", "missing value"], value=np.nan, inplace=True)

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')

    for col in df.columns:
        if df[col].dtype == "object":
            df[col].replace(to_replace=["na", "?", np.nan, "missing", "not available",
                                        "n/a", "missing value"], value="missing value", inplace=True)
            if df[col].unique().size < 25:
                considered_cols.append(col)
                keys[col] = {"values": df[col].unique().tolist(
                ), "start_i": last_index, "end_i": last_index + df[col].unique().size}
                last_index = last_index + df[col].unique().size
                for item in df[col].unique():
                    count.append([col, item])
    data_array = np.zeros((len(count), len(count)))

    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            for i in range(col+1, df.shape[1]):
                if df.columns[col] in considered_cols and df.columns[i] in considered_cols:
                    data_array[count.index([df.columns[col], df.iloc[row, col]]), count.index(
                        [df.columns[i], df.iloc[row, i]])] += 1

    data_array += np.transpose(data_array)
    return keys, data_array.tolist()

In [ ]:
def table_data_format(df_orig):
    metadata = []
    df = df_orig
    df.replace(to_replace=["na", "?", np.nan, "missing", "not available",
                           "n/a", "missing value"], value=np.nan, inplace=True)

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')

    for idx, col in enumerate(df.columns):
        helper = {}
        key = col
        if len(key) > 15:
            key = col[:15]

        helper['ascend'] = 0
        helper['name'] = key
        if(df.dtypes[col] == "object"):
            df[col].replace(to_replace=["na", "?", np.nan, "missing", "not available",
                                        "n/a", "missing value"], value="missing value", inplace=True)
            helper['datatype'] = 'string'
            helper['values'] = df[col].unique().tolist()
        elif(df.dtypes[col] == "int64"):
            helper['datatype'] = 'int'
            helper['min_val'] = df[col].min()
            helper['max_val'] = df[col].max()
        elif(df.dtypes[col] == "float64"):
            helper['datatype'] = 'float'
            helper['min_val'] = df[col].min()
            helper['max_val'] = df[col].max()
        metadata.append(helper)

    df.replace(to_replace=["na", "?", np.nan, "missing", "not available",
                           "n/a", "missing value"], value="missing value", inplace=True)
    data_array = []
    for row in range(df.shape[0]):
        aux = []
        for col in range(df.shape[1]):
            aux.append(df.iloc[row, col])
        data_array.append(aux)
    return metadata, data_array

In [65]:
fields = [
          'TP_SEXO', 'TP_COR_RACA', 'TP_ESCOLA', 'TP_DEPENDENCIA_ADM_ESC', 'TP_ENSINO', 
          'IN_TREINEIRO', 'TP_LOCALIZACAO_ESC', "TP_PRESENCA_CN", 
          "TP_PRESENCA_CH", "TP_PRESENCA_LC", "TP_PRESENCA_MT", "NU_NOTA_CN", 'NU_NOTA_CH', 
          'NU_NOTA_LC', 'NU_NOTA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_REDACAO', 'Q006', 'Q026',
          'IN_CEGUEIRA',
          'IN_DEFICIT_ATENCAO','IN_DISLEXIA',
          'IN_AUTISMO'
         ]
df = pd.read_csv('C:\\Users\\Lucas\\Downloads\\microdados_enem2018\\DADOS\\MICRODADOS_ENEM_2018.csv', 
                 sep=';', encoding='latin-1', skipinitialspace=True, usecols=fields)

df.drop(df[
    (df['IN_TREINEIRO'] == 1) |
    (df['TP_PRESENCA_CN'] != 1) |
    (df['TP_PRESENCA_CH'] != 1) |
    (df['TP_PRESENCA_LC'] != 1) |
    (df['TP_PRESENCA_MT'] != 1) |
    (df['TP_STATUS_REDACAO'] != 1) |
    (df['Q026'] != 'B')
    ].index, inplace=True)

df['NOTA_FINAL'] = list(map(lambda a,b,c,d, e: (a + b + c + d + e)/5, df['NU_NOTA_CN'], 
                            df['NU_NOTA_CH'], df['NU_NOTA_LC'], df['NU_NOTA_MT'], df['NU_NOTA_REDACAO']))
df["TP_COR_RACA"].replace(to_replace ={0:np.nan,1:"Branca",2:'Preta',3:"Parda", 4:'Amarela', 5:'Indígena'},
                          inplace=True)
df.drop(columns=['IN_TREINEIRO', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_PRESENCA_CN', 
                 'TP_STATUS_REDACAO', 'Q026', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 
                 'NU_NOTA_REDACAO'], inplace=True)
df["TP_ESCOLA"].replace(to_replace ={1:np.nan, 2:'Pública', 3:"Privada", 4:'Exterior'}, inplace=True)
df["TP_LOCALIZACAO_ESC"].replace(to_replace ={1:"Urbana", 2:'Rural'}, inplace=True)
df["TP_LINGUA"].replace(to_replace ={0:"Inglês", 1:'Espanhol'}, inplace=True)
df['TP_ENSINO'].replace(to_replace = {1: 'Regular', 2: 'Ed Especial', 3:'EJA'}, inplace=True)
df['Q006'].replace(to_replace = {'A' :'Nenhuma renda.','B' : '1 SM',
                                 'C' :'1 a 1.5 SM','D' : '1.5 a 2 SM',
                                 'E' :'2 a 2.5 SM','F' :'2.5 a 3 SM',
                                 'G' :'3 a 4 SM','H' :'4 a 5 SM',
                                 'I' :'5 a 6 SM','J' :'6 a 7 SM',
                                 'K' :'7 a 8 SM','L' :'8 a 9 SM',
                                 'M' :'9 a 10 SM','N' :'10 a 12 SM',
                                 'O' :'12 a 15 SM','P' :'15 a 20 SM',
                                 'Q' :'Mais de 20 SM'}, inplace=True)
df['TP_DEPENDENCIA_ADM_ESC'].replace(to_replace={1:'Federal', 2:'Estadual', 3:'Municipal', 4:'Privada'}, inplace=True)

q1,q2,q3 = df['NOTA_FINAL'].quantile([0.25,0.5,0.75])


df['NOTA_FINAL'] = pd.to_numeric(df['NOTA_FINAL'], errors='ignore')

def quartile_definition(val):
    if type(val) != 'str':
        if val < q1:
            return '1st quartile'
        if val < q2:
            return '2nd quartile'
        if val < q3:
            return '3rd quartile'

df['NOTA_FINAL'] = df['NOTA_FINAL'].map(quartile_definition)


# df['IN_BAIXA_VISAO'].replace(to_replace={1:'IN_BAIXA_VISAO', 0:'não'}, inplace=True)
df['IN_CEGUEIRA'].replace(to_replace={1:'IN_CEGUEIRA', 0:'não'}, inplace=True)
# df['IN_SURDEZ'].replace(to_replace={1:'IN_SURDEZ', 0:'não'}, inplace=True)
# df['IN_DEFICIENCIA_AUDITIVA'].replace(to_replace={1:'IN_DEFICIENCIA_AUDITIVA', 0:'não'}, inplace=True)
# df['IN_SURDO_CEGUEIRA'].replace(to_replace={1:'IN_SURDO_CEGUEIRA', 0:'não'}, inplace=True)
# df['IN_DEFICIENCIA_FISICA'].replace(to_replace={1:'IN_DEFICIENCIA_FISICA', 0:'não'}, inplace=True)
# df['IN_DEFICIENCIA_MENTAL'].replace(to_replace={1:'IN_DEFICIENCIA_MENTAL', 0:'não'}, inplace=True)
df['IN_DEFICIT_ATENCAO'].replace(to_replace={1:'IN_DEFICIT_ATENCAO', 0:'não'}, inplace=True)
df['IN_DISLEXIA'].replace(to_replace={1:'IN_DISLEXIA', 0:'não'}, inplace=True)
# df['IN_DISCALCULIA'].replace(to_replace={1:'IN_DISCALCULIA', 0:'não'}, inplace=True)
df['IN_AUTISMO'].replace(to_replace={1:'IN_AUTISMO', 0:'não'}, inplace=True)
# df['IN_VISAO_MONOCULAR'].replace(to_replace={1:'IN_VISAO_MONOCULAR', 0:'não'}, inplace=True)
# df['IN_OUTRA_DEF'].replace(to_replace={1:'IN_OUTRA_DEF', 0:'não'}, inplace=True)

df['ATENCAO_ESPECIAL'] = list(map(lambda a,b,c,d: (a + ' ' if a != 'não' else '') + (b + ' ' if b != 'não' else '') + (c + ' ' if c != 'não' else '') + (d + ' ' if d != 'não' else ''), df['IN_CEGUEIRA'], df['IN_DEFICIT_ATENCAO'] ,df['IN_DISLEXIA'] ,df['IN_AUTISMO']))
df['ATENCAO_ESPECIAL'].replace(to_replace={'': 'NP'}, inplace=True)
df.rename(columns={'Q06':'RENDA'})

,TP_SEXO,TP_COR_RACA,TP_ESCOLA,TP_ENSINO,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,IN_CEGUEIRA,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_AUTISMO,TP_LINGUA,Q006,NOTA_FINAL,ATENCAO_ESPECIAL
8,M,Parda,Pública,Regular,Estadual,Urbana,não,não,não,não,Inglês,1 a 1.5 SM,1st quartile,NP
16,M,Parda,Pública,Regular,Estadual,Urbana,não,não,não,não,Espanhol,1 SM,3rd quartile,NP
26,F,Branca,Pública,Regular,Estadual,Urbana,não,não,não,não,Inglês,1 SM,3rd quartile,NP
29,F,Parda,Pública,Regular,Estadual,Urbana,não,não,não,não,Espanhol,1 SM,1st quartile,NP
39,F,Branca,Pública,Regular,Estadual,Urbana,não,não,não,não,Inglês,1 a 1.5 SM,2nd quartile,NP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513730,M,Parda,Pública,Regular,Estadual,Urbana,não,não,não,não,Espanhol,2 a 2.5 SM,1st quartile,NP
5513735,F,Branca,NaN,Regular,NaN,NaN,não,não,não,não,Inglês,6 a 7 SM,1st quartile,NP
5513736,M,Parda,Pública,Regular,Estadual,Urbana,não,não,não,não,Inglês,1 SM,1st quartile,NP
5513743,F,Parda,Pública,Regular,Estadual,Urbana,não,não,não,não,Inglês,1 SM,1st quartile,NP


In [ ]:
file_loader = FileSystemLoader("templates")
env = Environment(loader=file_loader)

template = env.get_template("bid.html")

keys, data_array = build_data(df)

output = template.render(data=keys, full_matrix=data_array)

with open("result.html", "w") as result_file:
    result_file.write(output)